### Выполнила Зыкова Вероника, БКЛ-192

! Из данной версии первого дз удалена часть с клаулером, так как были взяты данные, собранные в предыдущий раз.

##### Импорт всех необходиимых модулей

In [1]:
import pymorphy2
from collections import Counter
from nltk.tokenize import word_tokenize
morph = pymorphy2.MorphAnalyzer()
from random import shuffle
import json

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vzyko\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Названия файлов

In [3]:
name_neg = 'neg_revs.jsonl'
name_pos = 'pos_revs.jsonl'

##### Предобработка данных

Читает файл с данными, где каждыя строка - json

In [4]:
def read_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = f.read()
    data = data.splitlines()
    texts = []
    for el in data:
        texts.extend(json.loads(el))
    return texts

Токенизация, очистка от пунктуации и прочих не-слов, лемматизация
- !NEW получение части речи

In [5]:
def clean_text(text):
    conn = {'ADJF': 'ADJ', 'ADJS': 'ADJ', 'COMP': 'ADJ', 'INFN': 'VERB', 'PRTF': 'ADJ', 'PRTS': 'VERB', 'GRND': 'VERB', 
            'NUMR': 'NUM', 'ADVB': 'ADV', 'NPRO':'PRON', 'PRED': 'ADV', 'PREP': 'ADP', 'PRCL': 'PART'}
    tokens = word_tokenize(text)
    lemmas = []
    for token in tokens:
        if token.isalpha():
            token_pr = morph.parse(token.lower())[0] 
            norm_form = token_pr.normal_form
            pos = str(token_pr.tag.POS)
            if pos in conn:
                pos = conn[pos]
            lemmas.append((norm_form, pos))
    return lemmas

Применяет предыдущую функцию ко всем текстам

In [6]:
def clean_all(reviews):
    clean_reviews = {}
    for mark in reviews:
        clean_reviews[mark] = []
        for el in reviews[mark]:
            text = clean_text(el)
            clean_reviews[mark].extend(text)
    return clean_reviews

Разделяет все тексты на тестовую и обучающую выборки, при этом уравнивая количество, если надо

In [7]:
def train_test(revs, perc=0.2, same=True):
    train = {}
    test = {}
    if same:
        line = min([len(revs[i]) for i in revs])
    for el in revs:
        if not same:
            line = len(revs[el])
        shuffle(revs[el])
        test[el] = revs[el][0:int(perc * line)]
        train[el] = revs[el][int(perc * line):line]
    return train, test

Читаем тексты, очищаем обучающую часть

In [8]:
revs = {}
revs['neg'] = read_file(name_neg)
revs['pos'] = read_file(name_pos)
train_text, test_text = train_test(revs)
train = clean_all(train_text)

Создаем множества, не включая в них слова с частотой меньше нужной

In [9]:
def get_sets(revs, freq_line=10):
    neg_set_full = set(revs['neg'])
    pos_set_full = set(revs['pos'])
    neg_freq = Counter(revs['neg'])
    pos_freq = Counter(revs['pos'])
    neg_set = neg_set_full - pos_set_full
    pos_set = pos_set_full - neg_set_full
    neg_set_tot = set([el for el in neg_set if neg_freq[el] >= freq_line])
    pos_set_tot = set([el for el in pos_set if pos_freq[el] >= freq_line])
    return neg_set_tot, pos_set_tot

- !NEW Сохраняем в отдельное множество биграммы, отвечающие шаблонам: не + ADJ, не + VERB, any + но 

In [10]:
def get_patterns(revs):
    res = set()
    for i in range(1, len(revs)):
        if revs[i - 1][0] == 'не':
            if revs[i][1] == 'VERB':
                res.add((revs[i - 1][0], revs[i][0]))
            elif revs[i][1] == 'ADJ':
                res.add((revs[i - 1][0], revs[i][0]))
        elif revs[i][0] == 'но':
            res.add((revs[i - 1][0], revs[i][0]))
    return res

In [11]:
def get_bigrams(revs):
    res = set()
    for i in range(1, len(revs)):
        res.add((revs[i - 1][0], revs[i][0]))
    return res

In [12]:
def get_all_patterns(revs, mode=get_patterns):
    negs = mode(revs['neg'])
    poss = mode(revs['pos'])
    negs_tot = negs - poss
    pos_tot = poss - negs
    return negs_tot, pos_tot

##### Проверка

Получаем количество слов, относящихся к негативному и позитивному классу соответственно

In [13]:
def get_score(text, neg, pos):
    text = clean_text(text)
    score = [0, 0]
    for el in text:
        if el in neg:
            score[0] += 1
        elif el in pos:
            score[1] += 1
    return tuple(score)

- !NEW Аналог предыдущей функции для биграмм

In [14]:
def get_score_patt(text, neg, pos):
    text = clean_text(text)
    score = [0, 0]
    for i in range(1, len(text)):
        if (text[i - 1], text[i]) in neg:
            score[0] += 1
        elif (text[i - 1], text[i]) in pos:
            score[1] += 1
    return tuple(score)

Получаем тип отзыва на основе результата предыдущей функции

In [15]:
def get_type(text, neg, pos, mode=get_score):
    score = mode(text, neg, pos)
    return score[1] > score[0]

Приводим выборку к нужному нам виду

In [16]:
def get_test(test):
    test_all = []
    marks = {'neg': 0, 'pos': 1}
    for mark in test:
        for el in test[mark]:
            test_all.append([el, marks[mark]])
    return test_all

Оцениваем определение тональности всех отзывов в выборке

In [17]:
def evaluate(data, neg, pos, mode=get_score):
    score = 0
    for el in data: 
        text_type = get_type(el[0], neg, pos, mode)
        if text_type == el[1]:
            score += 1
    return score / len(data)

Собираем множества, граница частотности указана в соотвествии с результатом перебора ниже

In [18]:
neg, pos = get_sets(train, freq_line=1)

In [19]:
test = get_test(test_text)
train_new = get_test(train_text)

- !NEW результаты для биграмм с шаблоном

In [20]:
neg_patt, pos_patt = get_all_patterns(train)

In [21]:
evaluate(train_new, neg_patt, pos_patt, mode=get_score_patt) # на обучающей

0.5

In [22]:
evaluate(test, neg_patt, pos_patt, mode=get_score_patt) # на тестовой

0.5

- !NEW результаты для биграмм

In [23]:
neg_bi, pos_bi = get_all_patterns(train, mode=get_bigrams)

In [24]:
evaluate(train_new, neg_bi, pos_bi, mode=get_score_patt) # на обучающей

0.5

In [25]:
evaluate(test, neg_bi, pos_bi, mode=get_score_patt) # на тестовой

0.5

Лучший из полученных результатов

In [26]:
evaluate(train_new, neg, pos) # на обучающей

1.0

In [27]:
evaluate(test, neg, pos) # на тестовой

0.7

Выводы: для работы с биграммами и тем более биграммами, построенными по шаблону, недостаточно данных в выборке, поэтому результат по сути оказывается равным тому, что мог бы выдать генератор случайных чисел. А модель со словами переобучается.